In [2]:
# Core PyTorch Library
import torch

In [3]:
# Neural Network modules (Layers, loss functions)
import torch.nn as nn

In [4]:
# Optimization algorithms (SGD, Adam, etc..)
import torch.optim as optim

In [5]:
# Utilities for loading data
from torch.utils.data import DataLoader

In [6]:
# torchvision provides datasets and image transformations
from torchvision import datasets, transforms

In [7]:
# For checking training progress
import os

In [3]:
# Use GPU if available (Kaggle provides GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print ("Using device", device)

Using device cpu


# Image Transformations
Neural networks need images to:
* Be the same size
* Be normalized
* Be converted to tensors

In [ ]:
# Define image preprocessing steps
transform = transforms.Compose([
    transforms.Resize((128, 128)), # Resize all images to 128 * 128
    transforms.ToTensor(), # Convert image to PyTorch tensor (C, H, W)
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5], # Normalize RGB channels
        std=[0.5, 0.5, 0.5]
    )
])

# Load Dataset Using ImageFolder
ImageFolder automatically assigns labels based on folder names.

In [1]:
import kagglehub
import os
import shutil

# Download dataset (this goes to kagglehub cache)
dataset_path = kagglehub.dataset_download("marquis03/cats-and-dogs")

print("Downloaded in cache at:", dataset_path)

# Your project directory
project_dir = r"C:\SUJEET\python-codes\pytorch-project\data"

# Create directory if not exists
os.makedirs(project_dir, exist_ok=True)

# Copy dataset from cache to your project directory
dataset_name = os.path.basename(dataset_path)
target_path = os.path.join(project_dir, dataset_name)

if not os.path.exists(target_path):
    shutil.copytree(dataset_path, target_path)
    print("Dataset copied to:", target_path)
else:
    print("Dataset already exists at:", target_path)


C:\SUJEET\SUJEET-GIT-Study\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Downloaded in cache at: C:\Users\TIAA USER.TIAABCP-32C0KP3.000\.cache\kagglehub\datasets\marquis03\cats-and-dogs\versions\2
Dataset already exists at: C:\SUJEET\python-codes\pytorch-project\data\2


In [4]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # resize all images to same size
    transforms.ToTensor(),           # convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],   # ImageNet normalization
                         std=[0.229, 0.224, 0.225])
])

# Dataset paths
train_dir = r"C:\SUJEET\python-codes\pytorch-project\data\train"
val_dir   = r"C:\SUJEET\python-codes\pytorch-project\data\val"

# Create datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset   = datasets.ImageFolder(root=val_dir, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Class names
class_names = train_dataset.classes
print("Classes:", class_names)

Classes: ['cat', 'dog']


# Define the CNN model
* 2 Convolutional layers
* 2 MaxPooling layers
* Fully connected layers

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # First convolutional Layer
        self.conv1 = nn.Conv2d(
            in_channels=3, # RGB image has 3 channels
            out_channels=16, # Number of filters
            kernel_size=3, # 3 * 3 filter
            padding=1 # Keep image size same
        )

        # Second convolutional layer
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)

        # Pooling layer (reduces image size)
        self.pool = nn.MaxPool2d(2, 2)

        # Fully connected Layers
        self.fc1 = nn.Linear(32 * 32 * 32, 128) # After polling
        self.fc2 = nn.Linear(128, 2) # 2 classes (cat, dog)

        # Activation
        self.relu = nn.ReLU()

    def forward(self, x):
        # Apply first convolutional + activation + pooling
        x = self.pool(self.relu(self.conv1(x)))
    
        # Apply second convolutional + activation + pooling
        x = self.pool(self.relu(self.conv2(x)))
    
        # Flatten tensor for fully connected layers
        x = x.view(x.size(0), -1)
    
        # Fully connected layer + activation
        x = self.relu(self.fc1(x))
    
        # Output layer (raw scores / logits)
        x = self.fc2(x)
    
        return x

# Initialize Model, Loss Function Optimizer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
# Create model and move it to device
model = SimpleCNN().to(device)

# Loss function for classification
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam is beginner-friendly)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
This is the heart of PyTorch training

In [ ]:
num_epochs = 5 # Increase Later for better accuracy

for epoch in range(num_epochs):
    running_loss = 0.0

    for images, labels in train_loader:
        # Move data GPU/CPU
        images = images.to(device)
        labels = labels.to(device)

        # Clear previous gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backprogation
        loss.backward()

        # Update weights
        optimizer.step()

        running_loss += loss.item()

    print (f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Simple Prediction Test

In [ ]:
# Switch model to evaluation mode
model.eval()

# Take one batch from training data
images, labels = next(iter(train_loader))
images, labels = images.to(device), labels.to(device)

# Disable gradient calculation
with torch.no_grad():
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)

print("Predicted labels:", predicted[:10].cpu().numpy())
print("True labels     :", labels[:10].cpu().numpy())